In [1]:

import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
import os

# from cycler import cycler


import matplotlib.pyplot as plt
import matplotlib as mpl
# import yfinance as yf
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
# import ccxt
from datetime import datetime, timedelta
import plotly.graph_objects as go
# import inspect 
# from IPython.display import display, HTML
# from tqdm import tqdm

In [12]:
df = pd.DataFrame({'A':[1,2,3],
                   'B':[4,5,6],
                   'C':[7,8,9],
                   'D':[1,3,5],
                   'E':[5,3,6],
                   'F':[7,4,3]})

# df = df.iloc[::-1] # reverse order

df = df.iloc[:, :-1] 

df
 

,A,B,C,D,E
0,1,4,7,1,5
1,2,5,8,3,3
2,3,6,9,5,6


In [ ]:

def get_historical_data(symbol, start_date = None):
    api_key = open(r'api_key.txt')
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={api_key}&outputsize=full'
    raw_df = requests.get(api_url).json()

    df = pd.DataFrame(raw_df[f'Time Series (Daily)']).T
    df = df.rename(columns = {'1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. adjusted close': 'adj close', '6. volume': 'volume'})
    
    for i in df.columns:
        df[i] = df[i].astype(float)
    
    df.index = pd.to_datetime(df.index)
    df = df.iloc[::-1].drop(['7. dividend amount', '8. split coefficient'], axis = 1)
    if start_date:
        df = df[df.index >= start_date]
    return df

googl = get_historical_data('GOOGL', '2020-01-01')
googl

In [6]:
df = pd.DataFrame(np.arange(20).reshape(5,4), columns=["A", "B", "C", "D"])
df

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [34]:
# Filter Pandas Dataframes

df = pd.DataFrame({
    'name':['Jane','John','Ashley','Mike','Emily','Jack','Catlin'],
    'ctg':['A','A','C','B','B','C','B'],
    'val':np.random.random(7).round(2),
    'val2':np.random.randint(1,10, size=7)
    })
df
# 	name	ctg	val	val2
# 0	Jane	A	0.09	5
# 1	John	A	0.32	9
# 2	Ashley	C	0.70	8
# 3	Mike	B	0.02	6
# 4	Emily	B	0.19	1
# 5	Jack	C	0.44	5
# 6	Catlin	B	0.59	8

df[df.val > 0.5]
# 	name	ctg	val	val2
# 2	Ashley	C	0.70	8
# 6	Catlin	B	0.59	8

df[df.name > 'Jane']
# 
df[(df.val > 0.5) & (df.val2 == 1)]
# 
df[(df.val < 0.5) | (df.val2 == 7)]
# 
names = ['John','Catlin','Mike']
df[df.name.isin(names)]
# 
df[df.name.str.startswith('J')]
# 
df[df.name.str.contains('y')]
# 
df[~df.name.str.startswith('J')]
# 	name	ctg	val	val2
# 2	Ashley	C	0.70	8
# 3	Mike	B	0.02	6
# 4	Emily	B	0.19	1
# 6	Catlin	B	0.59	8

df.query('ctg == "B" and val > 0.5')
# 
df.nlargest(3, 'val')
# 	name	ctg	val	val2
# 2	Ashley	C	0.70	8
# 6	Catlin	B	0.59	8
# 5	Jack	C	0.44	5

df.nsmallest(2, 'val2')
# 	name	ctg	val	val2
# 4	Emily	B	0.19	1
# 0	Jane	A	0.09	5

# The loc and iloc methods are used to select rows or columns based on index or label.
# loc: select rows or columns using labels
# iloc: select rows or columns using indices

df.iloc[3:5, :] #rows 3 and 4, all columns

#   name	ctg	 val   val2
# 3	Mike	B	0.59	5
# 4	Emily	B	0.63	1

# If the dataframe has integer index, the indices and labels of the rows are the same. 
# Thus, both loc and iloc accomplished the same thing on the rows.

df.loc[3:5, :] #rows 3 and 4 and 5, all columns
# 	name	ctg	val	val2
# 3	Mike	B	0.29	1
# 4	Emily	B	0.37	8
# 5	Jack	C	0.50	9

# if the index is not integers we canot use iloc



,name,ctg,val,val2
3,Mike,B,0.29,1
4,Emily,B,0.37,8
5,Jack,C,0.50,9


In [31]:
df[~df.name.str.startswith('J')]


,name,ctg,val,val2
2,Ashley,C,0.70,8
3,Mike,B,0.02,6
4,Emily,B,0.19,1
6,Catlin,B,0.59,8


In [ ]:
np.random.random(7).round(2)
# array([0.28, 0.21, 0.37, 0.9 , 0.68, 0.2 , 0.8 ])

np.random.random((7,2)).round(2)

# array([[0.63, 0.76],
#        [0.11, 0.48],
#        [0.03, 0.09],
#        [0.02, 0.78],
#        [0.3 , 0.3 ],
#        [0.27, 0.65],
#        [0.33, 0.43]])

np.random.randint(1,10, size=7) # >> size is the number of elements
# array([9, 2, 9, 9, 5, 9, 4])

np.random.randint(1,10, size=(7,2))
# array([[8, 5],
#        [4, 7],
#        [9, 4],
#        [7, 5],
#        [8, 5],
#        [2, 9],
#        [3, 1]])

In [ ]:
# How do I concatenate two lists in Python?

#########################
l1 = [1, 2, 3]
l2 = [4, 5, 6]

joinedlist = l1 + l2

#########################

# joining two lists 
# (applies to any iterable like sets, renges, tuples) 
# can now also be done with:


joined_list = [*l1, *l2]  # unpack both iterables in a list literal
print(joined_list)
# [1, 2, 3, 4, 5, 6]


# my_list + list(my_tuple) + list(my_range) 

# which is now equivalent to just 

# [*my_list, *my_tuple, *my_range]

# So while addition with + would raise a TypeError due to type mismatch:

# l = [1, 2, 3]
# r = range(4, 7)
# res = l + r
# The following won't:

# res = [*l, *r]

# because it will first unpack the contents of the iterables 
# and then simply create a list from the contents.

###################
# A nice example of the unpacking approach working on iterable types 
# is functions that return an iterator over one of the lists you're concatenating. 
# For example, you could reverse one of the lists you're concatenating: 

res = [*l1, *reversed(l2)] 

# Since reversed returns an iterator
res = l1 + reversed(l2) 
# would throw an error.

#################

# If you care about maintaining input order, then 

import collections 
mergedlist = list(collections.OrderedDict.fromkeys(l1 + l2))


# You can use sets to obtain merged list of unique values (remove duplicates)
mergedlist = list(set(l1 + l2))


In [ ]:
# same dict but diffrent style

dict1 = {'a':[1,2,3], 'b':[5,8,9]}

dict2 = dict(a = [1,2,3], b = [5,8,9])

dict3 = {**dict1, **dict2}

In [11]:
# conver str to datetime than convert datetime to unix time
# with the option of renaming columns

import pandas as pd
symbol = 'ETCUSDT'
df=pd.read_csv(f"C:\\Users\Grant\Desktop\work_git\work\data\\4h\{symbol}_4h.csv", parse_dates=True)
df.rename(columns={'Time': 'time', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close'}, inplace=True)

df['time'] = pd.to_datetime(df['time'])
df['time'] = (df['time'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df

,time,open,high,low,close,Volume
0,1631664000,57.14,57.49,56.28,56.86,163154.01
1,1631678400,56.86,57.46,56.40,57.43,138207.43
2,1631692800,57.41,58.91,57.39,57.93,364149.23
3,1631707200,57.93,58.94,57.50,58.86,249071.95
4,1631721600,58.87,59.06,58.37,58.54,142117.44
...,...,...,...,...,...,...
994,1645977600,28.92,28.95,27.09,27.13,182660.92
995,1645992000,27.13,27.80,26.89,27.36,294396.23
996,1646006400,27.36,27.86,26.89,27.29,183490.05
997,1646020800,27.30,27.82,26.91,27.68,177744.60


In [ ]:
# removing numbers which are close to each other in a list

# The below uses groupby to identify runs from 
# the iterable that start with a value start and 
# contain values that differ from start by no more than 4 
# We then collect all of those start values into a list.

from itertools import groupby

mylist = [75,75,76,77,78,79,154,155,154,156,260,262,263,550,551,551,552]

def runs(difference=4):
    start = None
    def inner(n):
        nonlocal start
        if start is None:
            start = n
        elif abs(start-n) > difference:
            start = n
        return start
    return inner

print([next(g) for k, g in groupby(mylist, runs())])
# [75, 154, 260, 550]

# This assumes that the input data is already sorted. 
# If it's not, you'll have to sort it: 

groupby(sorted(mylist), runs())

In [ ]:
# How to compare each item in a list with the rest, only once?

# itertools magic here to generate all possible combinations:

import itertools
mylist = range(5)
for x,y in itertools.combinations(mylist, 2):
    print(x,y)

# 0 1
# 0 2
# 0 3
# 0 4
# 1 2
# 1 3
# 1 4
# 2 3
# 2 4
# 3 4

# itertools.combinations will pair each element with
# each other element in the iterable, but only once.

# You could still write this using index-based item access, 
# equivalent to what you are used to, using nested for loops:

for i in range(len(mylist)):
    for j in range(i + 1, len(mylist)):
        print(mylist[i], mylist[j])

In [ ]:
# TypeError: '<' not supported between instances of 'Timestamp' and 'int'

# some where you have a comparison between a number and a timestap
# chnge the number into a time stamp 
# for example

num = pd.to_datetime(1490195805, unit='s')
# Timestamp('2017-03-22 15:16:45')

In [ ]:
# from unix to pd_datetime
df['time'] = pd.to_datetime(df['time'], unit='ms')

In [ ]:
df.reset_index(drop=True, inplace=True)


# drop : bool, default False
#     This resets the index to the default integer index.

# inplace : bool, default False
#     Modify the DataFrame in place (do not create a new object).

